In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import re
import string

from tensorflow.keras import layers
from tensorflow.keras import losses

# requires update to tensorflow 2.4
# >>> conda activate PIC16B
# >>> pip install tensorflow==2.4
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.layers.experimental.preprocessing import StringLookup

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Import Data

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/benbrill/MoodSpace/main/data/trainingSongs_clean.csv")
df.head()

,Unnamed: 0,trackName,artist,id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,lyrics,cluster,language
0,3,Overthinker,INZO,4K9xid96G3YmIvQZXN9SXg,0.472,0.605,8.0,-4.437,1.0,0.1340,0.03110,0.030800,0.1010,0.212,128.375,audio_features,a person who thinks all the time has nothing t...,0,en
1,4,Lifestyles of the Rich & Famous,Good Charlotte,2g2a5kDeZexbUTD8abcvm6,0.620,0.930,1.0,-3.685,1.0,0.0374,0.00043,0.000000,0.0686,0.609,106.220,audio_features,always see it on t v or read in the magazines ...,3,en
2,6,Carrying Your Love With Me,George Strait,7puxIVNdj5nsBJk43zM3bH,0.629,0.479,10.0,-10.608,1.0,0.0271,0.22000,0.000000,0.0587,0.345,138.231,audio_features,baby all i got is this beat up leather bag and...,5,en
3,7,"Check Yes, Juliet",We The Kings,0wVluBsVAVzBKrqspuCcwR,0.352,0.912,7.0,-4.253,1.0,0.0725,0.00197,0.000000,0.1930,0.351,166.795,audio_features,check yes juliet are you with me rain is falli...,2,en
4,8,At My Worst (feat. Kehlani),Pink Sweat$,58w68w4s8h9gw3xrDaXyuj,0.731,0.484,0.0,-5.579,1.0,0.0354,0.73000,0.000003,0.3260,0.350,92.043,audio_features,can i call you baby can you be my friend can y...,1,en


# Create Text Vectorization

In [3]:
max_tokens = 200
sequence_length = 500

vectorize_layer = TextVectorization(
    max_tokens=max_tokens, # only consider this many words
    output_mode='int',
    output_sequence_length=sequence_length) 

In [5]:
data = tf.data.Dataset.from_tensor_slices((df["lyrics"], df["cluster"]))

In [8]:
data = data.shuffle(buffer_size = len(data))

train_size = int(0.7*len(data))
val_size   = int(0.1*len(data))


train = data.take(train_size)
val   = data.skip(train_size).take(val_size)
test  = data.skip(train_size + val_size)
lyrics = train.map(lambda x, y: x)
vectorize_layer.adapt(lyrics)
len(train), len(val), len(test)

(501, 71, 145)

In [9]:
def vectorize_movie_script(text, label):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text), [label]

train_vec = train.map(vectorize_movie_script)
val_vec   = val.map(vectorize_movie_script)
test_vec  = test.map(vectorize_movie_script)

# Create Model

In [10]:
model = tf.keras.Sequential([
  layers.Embedding(max_tokens, output_dim = 10, name="embedding"),
  layers.Dropout(0.2),
  layers.GlobalAveragePooling1D(),
  layers.Dropout(0.2),
  layers.Dense(8)]
)

In [11]:
model.compile(loss=losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer='adam', 
              metrics=['accuracy'])

In [12]:
history = model.fit(train_vec, epochs = 100, validation_data = val_vec)

Epoch 1/100
501/501 [==============================] - 3s 4ms/step - loss: 2.0156 - accuracy: 0.2016 - val_loss: 1.9485 - val_accuracy: 0.2254
Epoch 2/100
501/501 [==============================] - 2s 4ms/step - loss: 1.9246 - accuracy: 0.2275 - val_loss: 1.8407 - val_accuracy: 0.3662
Epoch 3/100
501/501 [==============================] - 2s 4ms/step - loss: 1.9076 - accuracy: 0.2275 - val_loss: 1.9035 - val_accuracy: 0.2535
Epoch 4/100
501/501 [==============================] - 2s 4ms/step - loss: 1.8870 - accuracy: 0.2315 - val_loss: 1.7742 - val_accuracy: 0.2676
Epoch 5/100
501/501 [==============================] - 2s 4ms/step - loss: 1.8852 - accuracy: 0.2395 - val_loss: 1.9123 - val_accuracy: 0.1972
Epoch 6/100
501/501 [==============================] - 2s 4ms/step - loss: 1.8468 - accuracy: 0.2575 - val_loss: 1.8062 - val_accuracy: 0.2817
Epoch 7/100
501/501 [==============================] - 2s 4ms/step - loss: 1.8432 - accuracy: 0.2715 - val_loss: 1.7931 - val_accuracy: 0.3380

In [13]:
model.predict(test_vec)[1]

array([ 0.4165296 ,  0.86208546, -0.49146512, -0.5974111 , -0.37964135,
        0.7238439 , -0.97015285, -4.388351  ], dtype=float32)

In [11]:
model.evaluate(test_vec)

145/145 [==============================] - 0s 942us/step - loss: 1.8513 - accuracy: 0.2483


[1.8512661457061768, 0.24827586114406586]

In [12]:
model.save_weights('./checkpoint/my_checkpoint_10')

# Get Script Weights

In [15]:
import re
def stringProcessing(s):
    s = re.sub(r"\'", "", s)
    s = re.sub(r'\n', ' ', s)
    s = re.sub(r'\t', '', s)
    s = re.sub(r"\[[^[]*\]", '', s)
    s = re.sub(r'[^\w\s]', ' ', s)
    s = re.sub(r' +', ' ', s)
    s = s.strip()
    s = s.lower()
    return s

In [20]:
def vectorize_movie_script_2(text):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text)

for script_path in os.listdir("../scripts"):
    with open(f"../scripts/{script_path}") as f:
        contents = f.read()

        contents = stringProcessing(contents)

        df = pd.DataFrame({"lyrics": [contents]})
        


        data = tf.data.Dataset.from_tensor_slices((df["lyrics"]))
        vectorize_layer.adapt(data)
        data_vec = data.map(vectorize_movie_script_2)

        print(script_path, model.predict(data_vec)[0])

bourne.txt [ 0.9299726  -0.35914177 -0.51114357  0.6295208  -0.2103733   0.35397047
 -1.3569529  -5.057942  ]
deadpoets.txt [ 1.1206332  -0.18021777 -1.0099351  -0.4571914   0.6731483   0.242694
 -1.1417143  -5.3934536 ]
fellowship.txt [ 0.56383485  0.48283023 -0.6405089  -0.9706365   0.13684525  0.7688268
 -0.89722323 -5.0171576 ]
forrest.txt [ 0.97529155 -0.35290605 -0.54496884  0.5175109  -0.19030924  0.40814745
 -1.3905873  -5.1944103 ]
goodwillhunting.txt [ 0.88102204  0.0620276  -0.5795678  -0.5689312   0.27152842  0.4087851
 -1.0932559  -5.0554924 ]
incredibles.txt [ 0.8676267  -0.8479822  -0.10622385  0.93871295 -0.23126788  0.4184193
 -1.3599778  -4.8406067 ]
jedi.txt [ 0.55289084  0.6170999  -0.6333877  -0.66357017  0.25265342  0.225401
 -0.9125025  -4.9512897 ]
khan.txt [ 1.2249178e+00 -1.7303288e-01 -8.8581192e-01 -2.0883307e-03
 -2.2639140e-02  4.6894187e-01 -1.4066646e+00 -5.4672852e+00]
shawshank.txt [ 0.9405225   0.30564404 -0.96063006 -0.5697193   0.5954126   0.0423366